In [ ]:
import math

from portfolio import Portfolio
from backtest import Backtest
from strategies import *
from config import *
from utils import plot_distribution, PolygonAPI
from data_processing import *

In [ ]:
# get SPY data
spy_df = get_spy_data(start_date, end_date)
print("Number of days to backtest: ", len(spy_df))
spy_df.head(5)

In [ ]:
# instantiation
my_portfolio = Portfolio(initial_portfolio_nominal_value, portolio_weights_config, collateral_ratio)
my_strategy = ZeroCostCollar0DTE(my_portfolio, 'SPY', spy_df)
env = Backtest(my_portfolio, spy_df, PolygonAPI(api_key='MPkBRXXyfleZXSJQp8_bOsKuqo2Wi_Gk'))

# If price data is not available on Polygon.io, the BS model will be used to calculate price, which will print relevant info
if strategy_selected == 1:
    env.get_option_price('SPY', bs_config, zero_cost_search_config, open_price_config)
    my_strategy.find_zero_cost_collar(env)
elif strategy_selected == 2:
    my_strategy.select_options(env.main_df, strike_selection_config)
    env.get_option_price('SPY', bs_config)
else:
    raise ValueError("Check config, strategy does not exist. ")

env.update_option_price_at_expiration()
my_strategy.update_collar_pnl(env.main_df)

In [ ]:
# Buy and hold SPY at market open on Day 1
first_date = env.main_df['Date'].values[0]
first_price = env.main_df['Open'].values[0]

target_spy_exposure = my_portfolio.initial_portfolio_nominal_value * my_portfolio.target_portfolio_weights['equity']
n_spy_to_buy = math.floor(target_spy_exposure / first_price)
my_strategy.execute_buy_and_hold_underlying('equity', first_date, first_price, n_spy_to_buy)

print('Portfolio position:', my_portfolio.positions)
print('Cash after buying SPY:', my_portfolio.cash)
print('Transaction history')
my_portfolio.print_transaction_history()

In [ ]:
# Run the zero-cost collar strategy every day in spy_df
env.run(my_strategy)

In [ ]:
# Plot the distribution of collar strategy cost. Negative cost means the call premium is larger than put premium (i.e. receiving a credit to open)
plot_distribution(env.main_df['collar_cost'], x_label='Collar Cost', title='Collar Cost Distribution')

In [ ]:
# Plot the distribution collar strategy PnL
plot_distribution(env.main_df['collar_pnl'], x_label='Daily Collar Profit', title='Collar PnL Distribution')

In [ ]:
port_return = my_portfolio.calc_port_daily_return()
plot_distribution(port_return, x_label='Portfolio Daily Return', title='Portfolio Daily Return Distribution')

In [ ]:
spy_return = env.calc_benchmark_return()
plot_distribution(spy_return, x_label='SPY Daily Return', title='SPY Daily Return Distribution')

In [ ]:
env.plot_nav_vs_spy()

In [ ]:
env.plot_exposure()

In [ ]:
# Optional commands which could be useful for further review and analysis.

# Save the backtest main dataframe to the data folder
# env.save_main_df_to_csv()

# Save transaction history to the data folder
# my_portfolio.save_transaction_history_to_csv()

# Print all transaction history in the backtest
# my_portfolio.print_transaction_history()

# Print portfolio attributes
# print(my_portfolio.positions, my_portfolio.cash, my_portfolio.margin)
